<a href="https://colab.research.google.com/github/aoyingxue/Colab_Notebooks/blob/master/python_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
## put final_project under the directory of Colab Notebooks
import os
os.chdir("drive/My Drive/Colab Notebooks/final_project")

FileNotFoundError: ignored

In [0]:
!pip install pandasql

# Final Project

You will be asked to download data by yourselves and answer several data science questions through this final project. This project has two parts.


## Part 1 (15 Points):

This part contains 10 different data science questions that you are asked to answer. You need to download the corresponding data and write code to process the data to answer these questions. For each question, you need to provide your code to anser this question, as well as final answers. Each question will be worth 1.5 points in this part. 


## Final Submission:

Your final submission will contain two files:

1. The first would be this notebook. You need to provide code for your answers as well as your answers. You also need to provide visualization and executive summary using Markdown in this notebook. 

2. The second is the html version of this notebook. 


## Part 1

First, you need to download the most recent datasets containing listing and reviews in **Los Angeles** from [insidearibnb.com](http://insideairbnb.com/get-the-data.html). (September,2019,detailed data)You will then need to provide the code as well as answers to the following questions.

**To make sure that you downloaded the right dataset, your solution to Question 1 should be:**
1. **Number of unique listings = 45053**
2. **Number of unique hosts = 26286**

### Question 1: How many unique listings are there in the LA dataset? How many unique hosts are there?
**Question 1 Answer:**

1. Number of unique listings=45053
2. Number of unique hosts=26286

In [101]:
# You need to provide the code to answer the previous quesiton
# (you should also briefly explain your data structure -- 
# where do you put your data and how you get your data)

import pandas as pd
import numpy as np

listings_df=pd.read_csv("data/listings.csv")
print(len(listings_df.id.unique()))
print(len(listings_df.host_id.unique()))
reviews_df=pd.read_csv("data/reviews.csv")
calendar_df=pd.read_csv("data/calendar.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


45053
26286


### Question 2: What is the mean, median, standard deviation, minimum and maximum of the number of listings per host in LA? (for each host, you only need to consider the listings from LA)
**Question 2 Answer:**

1. Mean=12.375240
2. Median=1
3. Standard deviation=200.581428
4. Minimum=1
5. Maximum=18769

In [106]:
from pandasql import sqldf
def pysqldf(q):
    return sqldf(q, globals())

query1="""select host_id as host, sum(calculated_host_listings_count) as numoflisting \
          from listings_df where city="Los Angeles" group by host_id"""
df_q2=pysqldf(query1)
df_q2.numoflisting.describe()

count    16688.000000
mean        12.375240
std        200.581428
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max      18769.000000
Name: numoflisting, dtype: float64

### Question 3: What is the average and standard deviation number of listings for a super host versus a non-super host? Does super host or non-super host have more listings on average?
**Question 3 Answer:**


1. Average for a super host = 13.425567
2. Standard deviation for a super host = 355.529370 


3. Average for a non-super host = 11.354948
4. Standard deviation for a super host = 179.600226 

So super hosts have more listings on average.

In [107]:
# You need to provide the code to answer the previous quesiton
query2_non="""select sum(calculated_host_listings_count) as numoflisting\ 
              from listings_df\
              where host_is_superhost = "f"\
              group by host_id"""

query2_super="""select sum(calculated_host_listings_count) as numoflisting\
                from listings_df\
                where host_is_superhost = "t"\
                group by host_id"""

df_q3_non=pysqldf(query2_non)
df_q3_super=pysqldf(query2_super)
print(df_q3_non.numoflisting.describe())
print(df_q3_super.numoflisting.describe())

PandaSQLException: ignored

### Question 4: What are the unique types of host verifications are there? (It doesn't matter if you include "None" or empty string as a type)
**Question 4 Answer:**

There are 21 unique types of host verifications, including 'email', 'phone', 'facebook',  'reviews', 'kba', 'jumio', 'government_id', 'offline_government_id', 'selfie', 'identity_manual', 'work_email', 'manual_online', 'manual_offline', 'google', 'sent_id', 'None', 'sesame', 'sesame_offline', 'photographer', 'zhima_selfie', and 'weibo'.

In [108]:
## dataframe for host ids and verifications
host_verif=listings_df.drop_duplicates(
    keep="first",
    subset=["host_id","host_verifications"])[["host_id","host_verifications"]]

## deal with the characters
host_verif["host_verifications"]=[v.strip("[").strip("]").split(",")\
                                  for v in host_verif["host_verifications"]]
host_verif["host_verifications"]=host_verif["host_verifications"]\
                                .apply(lambda x: [s.strip("'").strip(" ")\
                                                  .lstrip("'") for s in x])

## verification types
verifications=[]
for i, item in enumerate(host_verif["host_verifications"]):
    if item not in verifications:
        for v in item:
            if v not in verifications and v!="":
                verifications.append(v)
print(verifications) 
print(len(verifications))

['email', 'phone', 'facebook', 'reviews', 'kba', 'jumio', 'government_id', 'offline_government_id', 'selfie', 'identity_manual', 'work_email', 'manual_online', 'manual_offline', 'google', 'sent_id', 'None', 'sesame', 'sesame_offline', 'photographer', 'zhima_selfie', 'weibo']
21


### Question 5: What are the five most popular verification types of hosts? For each type of the top five verifications, how many percent of hosts verify that type?
**Question 5 Answer:**

- The five most popular verification types of hosts are by phone (99.6766%), email (93.8370%), reviews (68.8541%), government_id (58.7689%), jumio (43.3387%).

In [109]:
## count how many times for each verification type to appear
veri_dict={}
for i,item in enumerate(host_verif["host_verifications"]):
    for j in range(len(item)):
        if item[j] not in veri_dict.keys():
            veri_dict[item[j]]=1
        else:
            veri_dict[item[j]]+=1 
## sort the times
sort_veri=sorted(((value,key) for (key,value) in veri_dict.items()),\
                 reverse=True)
top5_veri=[(key,value) for (value,key) in sort_veri[:5]]

## print the top 5 verifications and the times of each
top5_df=pd.concat([pd.DataFrame([[top5_veri[i][0],top5_veri[i][1]]],
                                columns=["type","times"])for i in range(5)])\
        .reset_index(drop=True)

## number of unique host ids
num_hosts=host_verif.host_id.nunique()
print(num_hosts) ## total number of hosts

top5_df["perc"]=top5_df.times.apply(lambda x:x/num_hosts)
top5_df

26286


,type,times,perc
0,phone,26201,0.996766
1,email,24666,0.938370
2,reviews,18099,0.688541
3,government_id,15448,0.587689
4,jumio,11392,0.433387


### Question 6: What is the mean, standard deviation of the average price of a listing in Los Angles from 01/01/2020 to 03/01/2020 (inclusive)? (For listings that are not available in this time, you should not count them in the average price.)

**Question 6 Answer:**

- Mean = 218.028763
- Standard deviation = 580.214329

In [110]:
## process price column
calendar_df.price=calendar_df.price.apply(lambda x: float(x.strip("$")\
                                                          .replace(",",""))\
                                          if type(x)==str else x)

## process date column and filter
calendar_df.date=pd.to_datetime(calendar_df.date)
calendar_temp=calendar_df.loc[(calendar_df.date>=pd.Timestamp(2020,1,1)) & \
                              (calendar_df.date<=pd.Timestamp(2020,3,1)) & \
                              calendar_df.price!="f",]\
                .reset_index(inplace=False,drop=True)


## group by listing ids and calculate average price
avg_price_df=calendar_temp.groupby(by="listing_id").price.mean()
print(avg_price_df.describe())


/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


count    45053.000000
mean       218.028763
std        580.214329
min         10.027397
25%         71.531507
50%        111.446575
75%        190.493151
max      25000.000000
Name: price, dtype: float64


### Question 7: For reviews that are written in Jan, 2016, what are the most popular ten words except stopping words? What about reviews written in March, 2017 and April, 2018?
**Hint**: You need to use NTLK stopping words to find the stopping words in Python. You also need to remove punctuation and numbers

**Question 7 Answer:**

- Top 10 popular words for Jan 2016 are ('great', 5374), ('stay', 4661), ('place', 4284), ('clean', 2719), ('location', 2694), ('house', 2642), ('us', 2630), ('nice', 2403), ('would', 2382), ('host', 2325).

- Top 10 popular words for March 2017 are ('great', 12169), ('place', 11528), ('stay', 9453), ('location', 5860), ('clean', 5444), ('nice', 4540), ('would', 4514), ('host', 4422), ('house', 3769), ('comfortable', 3756).

- Top 10 popular words for April 2018 are ('great', 18787), ('place', 18654), ('stay', 14739), ('location', 9983), ('clean', 9031), ('would', 7263), ('nice', 6797), ('host', 6789), ('house', 5516), ('la', 5491)


In [111]:
## import and download nltk stopwords
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
import string

## process date column and filter
reviews_df.date=pd.to_datetime(reviews_df.date)

def find_top_word(reviews_df,month_num,year_num):
    if month_num<12:
        reviews=reviews_df.loc[(reviews_df.date>=pd.Timestamp(year_num,month_num,1)) & \
                               (reviews_df.date<pd.Timestamp(year_num,month_num+1,1)),]\
                               .reset_index(inplace=False,drop=True)
        reviews.comments.dropna(axis=0)

        ## process punctuations
        transtable = str.maketrans('', '', string.punctuation) ## the third argument will be mapped to None
        comments=reviews.comments.apply(lambda x:x.lower()\
                                              .translate(transtable)\
                                              .split(" ") if type(x)==str else x)
        ## remove stop words
        stop_words=set(stopwords.words('english'))
        comments=comments.apply(lambda x: [s for s in x if ((s not in stop_words)\
                                                            & (s!=''))]\
                                ## omit nan
                                if type(x)==list else x)

        ## count different words
        all_words={}
        for i,item in comments.iteritems():
            if type(item)==list: ## omit nan
                for w in item:
                   if w not in all_words:
                      all_words[w]=1
                   else:
                     all_words[w]+=1
            

        ## sort by key
        sort_words=sorted(((value,key) for (key,value) in all_words.items()),\
                         reverse=True)
        top_words=[(key,value) for (value,key) in sort_words[0:10]]
        return top_words
print("Top words for Jan 2016 are",find_top_word(reviews_df,1,2016))
print("Top words for March 2017 are",find_top_word(reviews_df,3,2017))
print("Top words for March 2017 are",find_top_word(reviews_df,4,2018))

Top words for Jan 2016 are [('great', 5374), ('stay', 4661), ('place', 4284), ('clean', 2719), ('location', 2694), ('house', 2642), ('us', 2630), ('nice', 2403), ('would', 2382), ('host', 2325)]
Top words for March 2017 are [('great', 12169), ('place', 11528), ('stay', 9453), ('location', 5860), ('clean', 5444), ('nice', 4540), ('would', 4514), ('host', 4422), ('house', 3769), ('comfortable', 3756)]
Top words for March 2017 are [('great', 18787), ('place', 18654), ('stay', 14739), ('location', 9983), ('clean', 9031), ('would', 7263), ('nice', 6797), ('host', 6789), ('house', 5516), ('la', 5491)]


### Question 8: Which five zipcode has the highest average listing price and has at least 30 listings (again only considering available dates) between 2020-01-01 to 2020-02-01 (inclusive)? What are these listing prices? What are the number of active listings and number of hosts in these top five zipcodes (a listing is active if it has at least one available date in the calendar data)?

**Question 8 Answer:**

- Top 5 zipcodes that have the highest average listing price are 90210, 90077, 90265, 90069, 91436. 
- The average listing prices are \$1963.6, \$1909.1, \$1559.6, \$998.1, \$742.7. 
- Numbers of active listings are 278, 74, 329, 423, 40.
- Numbers of hosts are 173, 48, 221, 305, 33.

In [114]:
 ## filter dates
calendar_df.date=pd.to_datetime(calendar_df.date)
calendar_temp=calendar_df.loc[(calendar_df.date>=pd.Timestamp(2020,1,1)) & \
                               (calendar_df.date<=pd.Timestamp(2020,2,1)),]\
                               .reset_index(inplace=False,drop=True)

calendar_temp=calendar_temp.loc[calendar_temp.available!='f',]

## find 
query8="select t1.zipcode as zipcode, \
        count(distinct t1.id) as listings, \
        sum(t2.sum_price)/sum(t2.count_price) as avg_price, \
        count(distinct host_id) as hosts \
        \
        from listings_df as t1 join \
        \
        (select listing_id, sum(price) as sum_price, count(price) as count_price \
        from calendar_temp \
        group by listing_id) as t2 \
        \
        on t1.id=t2.listing_id \
        group by t1.zipcode \
        having listings>=30 \
        order by avg_price desc limit 5"

df_q8=pysqldf(query8)
print(df_q8)

  zipcode  listings    avg_price  hosts
0   90210       278  1963.557367    173
1   90077        74  1909.066130     48
2   90265       329  1559.605010    221
3   90069       423   998.113153    305
4   91436        40   742.660019     33


### Question 9: Focusing on the data from 2020-01-01 to 2020-04-01 (inclusive), what are five zipcodes that have at least 30 listings and have the largest absolute difference between the average prices on weekends versus the average prices on weekdays? (Weekends = Saturday and Sunday). Please give the zipcodes as well as the listing prices
**Question 9 Answer:**

- Top 5 zipcodes are 90265, 91325, 91436, 90293, 90048.
- The 5 largest absolute differences between prices on weekends and weekdays are 24.7, 15.3, 12.2, 12.1, 10.4.
- The average prices on weekends and weekdays are presented in the table.

In [0]:
## process date and weekday
calendar_df["weekday"]=calendar_df.date.dt.weekday
calendar_temp2=calendar_df.loc[(calendar_df.date>=pd.Timestamp(2020,1,1)) & \
                               (calendar_df.date<=pd.Timestamp(2020,4,1)),]\
                               .reset_index(inplace=False,drop=True)
## find 
query9="select t1.zipcode as zipcode, \
        count(distinct t1.id) as listings, \
        sum(t2.sum_weekend_price)/sum(t2.count_weekend_price) as avg_weekend_price, \
        sum(t2.sum_weekday_price)/sum(t2.count_weekday_price) as avg_weekday_price, \
        abs(sum(t2.sum_weekend_price)/sum(t2.count_weekend_price)-sum(t2.sum_weekday_price)/sum(t2.count_weekday_price)) as difference \
        \
        from listings_df as t1 join \
        \
        ((select listing_id, sum(price) as sum_weekend_price, count(price) as count_weekend_price \
        from calendar_temp2 \
        where available!='f' and (weekday=5 or weekday=6)\
        group by listing_id) join \
        (select listing_id, sum(price) as sum_weekday_price, count(price) as count_weekday_price \
        from calendar_temp2 \
        where available!='f' and (weekday!=5 and weekday!=6)\
        group by listing_id) using (listing_id)) as t2 \
        \
        on t1.id=t2.listing_id \
        group by t1.zipcode \
        having listings>=30 \
        order by difference desc limit 5"
df_q9=pysqldf(query9)
print(df_q9)

  zipcode  listings  avg_weekend_price  avg_weekday_price  difference
0   90265       327        1584.743035        1560.047831   24.695205
1   91325        51         318.357696         303.016354   15.341341
2   91436        40         717.131461         704.904720   12.226741
3   90293        91         286.943370         274.821375   12.121994
4   90048       410         426.101584         415.712282   10.389302


### Question 10: What is the average and standard deviation of the daily total capacity at Airbnb Los Angeles from 2020-01-01 to 2020-04-01? The daily total capacity in a day is the number of beds that are available in that given date. What is the average and standard deviation of the daily price per bed at Airbnb Los Angeles from 2020-01-01 to 2020-04-01?
**Question 10 Answer:**

- The average of the daily total capacity is 46995.228261.
- The standard deviation of the daily total capacity is 4980.641316.
- The average of the daily price per bed is 454.513200.
- The standard deviation of the daily price per bed is 24.440251.

In [0]:
## already filtered out the dates in the previous question
## filter out the unavailable dates and ids
calendar_temp3=calendar_temp2.loc[calendar_temp2.available!='f',]

## query to calculate the total capacity each day
query10="select t2.date as date, sum(t1.beds) as beds \
from calendar_temp3 as t2 join listings_df as t1 on t1.id=t2.listing_id \
group by date"

df_q10=pysqldf(query10)
print(df_q10)

## print out the mean and standard deviation of daily capacity
print(df_q10.describe())

                          date     beds
0   2020-01-01 00:00:00.000000  44519.0
1   2020-01-02 00:00:00.000000  46206.0
2   2020-01-03 00:00:00.000000  47415.0
3   2020-01-04 00:00:00.000000  48256.0
4   2020-01-05 00:00:00.000000  48660.0
..                         ...      ...
87  2020-03-28 00:00:00.000000  38157.0
88  2020-03-29 00:00:00.000000  38203.0
89  2020-03-30 00:00:00.000000  38214.0
90  2020-03-31 00:00:00.000000  38313.0
91  2020-04-01 00:00:00.000000  38221.0

[92 rows x 2 columns]
               beds
count     92.000000
mean   46995.228261
std     4980.641316
min    37768.000000
25%    47112.750000
50%    49507.000000
75%    49979.500000
max    50863.000000


In [0]:
## calculate daily price
## should it be sum(beds of each id * price of each id)/total number of beds?
query10_2="select t3.date, sum(t3.weighted_total_price) as sum_price from \
(select t1.id as id, t2.date as date, sum(t1.beds)*t2.price as weighted_total_price \
from listings_df as t1 join calendar_temp3 as t2 on t1.id=t2.listing_id \
group by t1.id, t2.date) as t3 \
group by t3.date"
df_q10_2=pysqldf(query10_2)

## bind two dataframes and calculate the daily price per bed each day
df_q10_3=pd.DataFrame({"date":df_q10.date,"price_per_bed":df_q10_2.sum_price/df_q10.beds})
print(df_q10_3)
print(df_q10_3.describe())

                          date  price_per_bed
0   2020-01-01 00:00:00.000000     521.028011
1   2020-01-02 00:00:00.000000     496.470329
2   2020-01-03 00:00:00.000000     497.247137
3   2020-01-04 00:00:00.000000     490.819649
4   2020-01-05 00:00:00.000000     464.637752
..                         ...            ...
87  2020-03-28 00:00:00.000000     501.949131
88  2020-03-29 00:00:00.000000     484.430071
89  2020-03-30 00:00:00.000000     481.930549
90  2020-03-31 00:00:00.000000     481.654817
91  2020-04-01 00:00:00.000000     482.054656

[92 rows x 2 columns]
       price_per_bed
count      92.000000
mean      454.513200
std        24.440251
min       432.222202
25%       434.221409
50%       447.293050
75%       481.961576
max       521.028011


## Part 2 (15 Points):

In this part, you need to find **5** interesting business questions out of the datasets. You then need to write Python code to answer the questions. Last, you need to write a 200-word summary of your answers and business insights you get from ansewr these 5 questions based on your code. You will be evaluated based on the following criterion:

1. You need to ask three business-relevant questions (2 points)
2. You need to answer these three questions using Python and the datasets (1 points)
3. You have at least **5** graphs to visualize your insights (2 points)
4. Your executive summary of your answers are well-written and make sense (5 points)
5. Your business insights are very interesting and the report is creative (5 points)


**(You need to use this cell to write your 5 questions)**

**Question 1:**

**Question 2:**

**Question 3:**

**Question 4:**

**Question 5:**


In [0]:
# Your code to answer Question 1

In [0]:
# Your code to answer Question 2

In [0]:
# Your code to answer Question 3

In [0]:
# Your code to answer Question 4

In [0]:
# Your code to answer Question 5

## Executive Summary
**(You need to use this cell to write your executive summary)**
